In [1]:
#!/usr/bin/env python
# coding: utf-8


#libraries

import pandas as pd
import numpy  as np 
import pickle
import joblib
import http.client
import json
import time
import csv
from   datetime import datetime, timedelta
import holidays
import sqlite3
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Configure Nrgstrem API call functionalities

class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)        

# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')      


# create the dataframe to hold input data for 24-h forecast

column_list = ['year', 'day_of_month', 'month',
       'day_of_year', 'hour_of_day', 'hour_of_the_week', 'hour_x_day',
       'sin.hour.daily', 'cos.hour.daily', 'sin.hour.weekly',
       'cos.hour.weekly', 'sin.day.yearly', 'cos.day.yearly', 'weekend',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
       'sunday', 'temp_calgary', 'temp_edmonton', 'is_Covid',
       'temp_edmonton_lag1', 'temp_edmonton_lag2', 'temp_edmonton_lag3',
       'temp_edmonton_lag4', 'temp_edmonton_lag5', 'temp_edmonton_lag6',
       'temp_edmonton_lag7', 'temp_edmonton_lag8', 'temp_edmonton_lag9',
       'temp_edmonton_lag10', 'temp_edmonton_lag11', 'temp_edmonton_lag12',
       'temp_edmonton_lag13', 'temp_edmonton_lag14', 'temp_edmonton_lag15',
       'temp_edmonton_lag16', 'temp_edmonton_lag17', 'temp_edmonton_lag18',
       'temp_edmonton_lag19', 'temp_edmonton_lag20', 'temp_edmonton_lag21',
       'temp_edmonton_lag22', 'temp_edmonton_lag23', 'temp_edmonton_lag24',
       'temp_calgary_lag1', 'temp_calgary_lag2', 'temp_calgary_lag3',
       'temp_calgary_lag4', 'temp_calgary_lag5', 'temp_calgary_lag6',
       'temp_calgary_lag7', 'temp_calgary_lag8', 'temp_calgary_lag9',
       'temp_calgary_lag10', 'temp_calgary_lag11', 'temp_calgary_lag12',
       'temp_calgary_lag13', 'temp_calgary_lag14', 'temp_calgary_lag15',
       'temp_calgary_lag16', 'temp_calgary_lag17', 'temp_calgary_lag18',
       'temp_calgary_lag19', 'temp_calgary_lag20', 'temp_calgary_lag21',
       'temp_calgary_lag22', 'temp_calgary_lag23', 'temp_calgary_lag24']
# generate time-stamps for forecast hours

now_time                 = datetime.now()
year_of_run              = now_time.year
month_of_run             = now_time.month
day_of_run               = now_time.day
forecast_start           = datetime(year_of_run ,month_of_run , day_of_run)
forecast_interval_stamps = pd.date_range(forecast_start, periods=24, freq= 'h')

prediction_df            = pd.DataFrame(columns = column_list)

In [2]:
prediction_df['day_of_month']  = forecast_interval_stamps.day
prediction_df['month']         = forecast_interval_stamps.month

day = pd.Series([d.timetuple().tm_yday for d in forecast_interval_stamps])
prediction_df['day_of_year'] = day

hrs                          = np.arange(0, 24, 1).tolist()
prediction_df['hour_of_day'] = hrs

prediction_df['hour_of_the_week'] = forecast_interval_stamps.dayofweek * 24 + (forecast_interval_stamps.hour + 1)

# add time-related features


prediction_df['hour_x_day'] = prediction_df['day_of_year']*prediction_df['hour_of_day'] 

prediction_df['sin.hour.daily']  = np.sin(prediction_df['hour_of_day']*2*np.pi/24)
prediction_df['cos.hour.daily']  = np.cos(prediction_df['hour_of_day']*2*np.pi/24)

prediction_df['sin.hour.weekly'] = np.sin(prediction_df['hour_of_the_week']*2*np.pi/168 )
prediction_df['cos.hour.weekly'] = np.cos(prediction_df['hour_of_the_week']*2*np.pi/168 )

prediction_df['sin.day.yearly'] = np.sin(prediction_df['day_of_year']*2*np.pi/365 + 3*np.pi/2)
prediction_df['cos.day.yearly'] = np.cos(prediction_df['day_of_year']*2*np.pi/365 + 3*np.pi/2)

#

weekdays                  = [d.weekday() for d in forecast_interval_stamps]
prediction_df['weekend']  = [1 if d >= 5 else 0 for d in weekdays]

name_of_day = forecast_start.strftime("%A")
for i in range(14,14+7):
    if name_of_day.lower() == prediction_df.columns[i]:
        prediction_df.iloc[:,i] = 1
    else:
        prediction_df.iloc[:,i] = 0

prediction_df['year'] = year_of_run-2000
prediction_df.iloc[:,0:21]

,year,day_of_month,month,day_of_year,hour_of_day,hour_of_the_week,hour_x_day,sin.hour.daily,cos.hour.daily,sin.hour.weekly,...,sin.day.yearly,cos.day.yearly,weekend,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,21,25,2,56,0,73,0,0.000000e+00,1.000000e+00,3.998920e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
1,21,25,2,56,1,74,56,2.588190e-01,9.659258e-01,3.653410e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
2,21,25,2,56,2,75,112,5.000000e-01,8.660254e-01,3.302791e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
3,21,25,2,56,3,76,168,7.071068e-01,7.071068e-01,2.947552e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
4,21,25,2,56,4,77,224,8.660254e-01,5.000000e-01,2.588190e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
5,21,25,2,56,5,78,280,9.659258e-01,2.588190e-01,2.225209e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
6,21,25,2,56,6,79,336,1.000000e+00,6.123234e-17,1.859116e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
7,21,25,2,56,7,80,392,9.659258e-01,-2.588190e-01,1.490423e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
8,21,25,2,56,8,81,448,8.660254e-01,-5.000000e-01,1.119645e-01,...,-0.570242,0.821477,0,0,0,0,1,0,0,0
9,21,25,2,56,9,82,504,7.071068e-01,-7.071068e-01,7.473009e-02,...,-0.570242,0.821477,0,0,0,0,1,0,0,0


In [3]:
# weather feature

fromDate = '{}/{}/{}'.format(month_of_run,day_of_run, year_of_run )
toDate   = fromDate

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_list = {}

streams = [242498, 242497]
city_names = ['calgary', 'edmonton']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    
    df = df[16:(16+24)] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction','Datetime', 'wind'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]

alberta_weather_merged = pd.concat([df_clgry, df_edmtn], axis=1)
alberta_weather_merged['temp_calgary'] = alberta_weather_merged['temp_calgary'].astype('float64')
alberta_weather_merged['temp_edmonton'] = alberta_weather_merged['temp_edmonton'].astype('float64')
prediction_df['temp_calgary'] = alberta_weather_merged.iloc[:,0].values
prediction_df['temp_edmonton'] = alberta_weather_merged.iloc[:,1].values

prediction_df.iloc[:,21:23]

2021-02-25 12:18:33.889913 Outputing stream /api/StreamData/242498?fromDate=2/25/2021&toDate=2/25/2021 res code 200
2021-02-25 12:18:35.546729 Outputing stream /api/StreamData/242497?fromDate=2/25/2021&toDate=2/25/2021 res code 200


,temp_calgary,temp_edmonton
0,-4.0,-6.0
1,-5.0,-6.0
2,-6.0,-7.0
3,-5.0,-6.0
4,-3.0,-4.0
5,-2.0,-3.0
6,-2.0,-2.0
7,-1.0,-2.0
8,-1.0,-1.0
9,0.0,0.0


In [4]:
prediction_df['is_Covid'] = 0

In [5]:
prediction_df['temp_edmonton_lag1'] = prediction_df['temp_edmonton'].shift(1)
prediction_df['temp_edmonton_lag2'] = prediction_df['temp_edmonton'].shift(2)
prediction_df['temp_edmonton_lag3'] = prediction_df['temp_edmonton'].shift(3)
prediction_df['temp_edmonton_lag4'] = prediction_df['temp_edmonton'].shift(4)
prediction_df['temp_edmonton_lag5'] = prediction_df['temp_edmonton'].shift(5)
prediction_df['temp_edmonton_lag6'] = prediction_df['temp_edmonton'].shift(6)
prediction_df['temp_edmonton_lag7'] = prediction_df['temp_edmonton'].shift(7)
prediction_df['temp_edmonton_lag8'] = prediction_df['temp_edmonton'].shift(8)
prediction_df['temp_edmonton_lag9'] = prediction_df['temp_edmonton'].shift(9)
prediction_df['temp_edmonton_lag10'] = prediction_df['temp_edmonton'].shift(10)
prediction_df['temp_edmonton_lag11'] = prediction_df['temp_edmonton'].shift(11)
prediction_df['temp_edmonton_lag12'] = prediction_df['temp_edmonton'].shift(12)
prediction_df['temp_edmonton_lag13'] = prediction_df['temp_edmonton'].shift(13)
prediction_df['temp_edmonton_lag14'] = prediction_df['temp_edmonton'].shift(14)
prediction_df['temp_edmonton_lag15'] = prediction_df['temp_edmonton'].shift(15)
prediction_df['temp_edmonton_lag16'] = prediction_df['temp_edmonton'].shift(16)
prediction_df['temp_edmonton_lag17'] = prediction_df['temp_edmonton'].shift(17)
prediction_df['temp_edmonton_lag18'] = prediction_df['temp_edmonton'].shift(18)
prediction_df['temp_edmonton_lag19'] = prediction_df['temp_edmonton'].shift(19)
prediction_df['temp_edmonton_lag20'] = prediction_df['temp_edmonton'].shift(20)
prediction_df['temp_edmonton_lag21'] = prediction_df['temp_edmonton'].shift(21)
prediction_df['temp_edmonton_lag22'] = prediction_df['temp_edmonton'].shift(22)
prediction_df['temp_edmonton_lag23'] = prediction_df['temp_edmonton'].shift(23)
prediction_df['temp_edmonton_lag24'] = prediction_df['temp_edmonton'].shift(24)

prediction_df['temp_calgary_lag1'] = prediction_df['temp_calgary'].shift(1)
prediction_df['temp_calgary_lag2'] = prediction_df['temp_calgary'].shift(2)
prediction_df['temp_calgary_lag3'] = prediction_df['temp_calgary'].shift(3)
prediction_df['temp_calgary_lag4'] = prediction_df['temp_calgary'].shift(4)
prediction_df['temp_calgary_lag5'] = prediction_df['temp_calgary'].shift(5)
prediction_df['temp_calgary_lag6'] = prediction_df['temp_calgary'].shift(6)
prediction_df['temp_calgary_lag7'] = prediction_df['temp_calgary'].shift(7)
prediction_df['temp_calgary_lag8'] = prediction_df['temp_calgary'].shift(8)
prediction_df['temp_calgary_lag9'] = prediction_df['temp_calgary'].shift(9)
prediction_df['temp_calgary_lag10'] = prediction_df['temp_calgary'].shift(10)
prediction_df['temp_calgary_lag11'] = prediction_df['temp_calgary'].shift(11)
prediction_df['temp_calgary_lag12'] = prediction_df['temp_calgary'].shift(12)
prediction_df['temp_calgary_lag13'] = prediction_df['temp_calgary'].shift(13)
prediction_df['temp_calgary_lag14'] = prediction_df['temp_calgary'].shift(14)
prediction_df['temp_calgary_lag15'] = prediction_df['temp_calgary'].shift(15)
prediction_df['temp_calgary_lag16'] = prediction_df['temp_calgary'].shift(16)
prediction_df['temp_calgary_lag17'] = prediction_df['temp_calgary'].shift(17)
prediction_df['temp_calgary_lag18'] = prediction_df['temp_calgary'].shift(18)
prediction_df['temp_calgary_lag19'] = prediction_df['temp_calgary'].shift(19)
prediction_df['temp_calgary_lag20'] = prediction_df['temp_calgary'].shift(20)
prediction_df['temp_calgary_lag21'] = prediction_df['temp_calgary'].shift(21)
prediction_df['temp_calgary_lag22'] = prediction_df['temp_calgary'].shift(22)
prediction_df['temp_calgary_lag23'] = prediction_df['temp_calgary'].shift(23)
prediction_df['temp_calgary_lag24'] = prediction_df['temp_calgary'].shift(24)

prediction_df.iloc[:,24:72]

,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3,temp_edmonton_lag4,temp_edmonton_lag5,temp_edmonton_lag6,temp_edmonton_lag7,temp_edmonton_lag8,temp_edmonton_lag9,temp_edmonton_lag10,...,temp_calgary_lag15,temp_calgary_lag16,temp_calgary_lag17,temp_calgary_lag18,temp_calgary_lag19,temp_calgary_lag20,temp_calgary_lag21,temp_calgary_lag22,temp_calgary_lag23,temp_calgary_lag24
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-6.0,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-7.0,-6.0,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.0,-7.0,-6.0,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-4.0,-6.0,-7.0,-6.0,-6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
datetime_pastday   = now_time + pd.Timedelta(days= -1)
fromDate_past      = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate_past        = fromDate_past

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_list = {}

streams = [242498, 242497]
city_names = ['calgary', 'edmonton']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    
    df = df[16:(16+24)] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction','Datetime', 'wind'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]

alberta_weather_merged = pd.concat([df_clgry, df_edmtn], axis=1)
alberta_weather_merged['temp_calgary'] = alberta_weather_merged['temp_calgary'].astype('float64')
alberta_weather_merged['temp_edmonton'] = alberta_weather_merged['temp_edmonton'].astype('float64')
alberta_weather_merged

2021-02-25 12:18:37.135424 Outputing stream /api/StreamData/242498?fromDate=2/25/2021&toDate=2/25/2021 res code 200
2021-02-25 12:18:39.664367 Outputing stream /api/StreamData/242497?fromDate=2/25/2021&toDate=2/25/2021 res code 200


,temp_calgary,temp_edmonton
0,-4.0,-6.0
1,-5.0,-6.0
2,-6.0,-7.0
3,-5.0,-6.0
4,-3.0,-4.0
5,-2.0,-3.0
6,-2.0,-2.0
7,-1.0,-2.0
8,-1.0,-1.0
9,0.0,0.0


In [7]:
prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag1')] = alberta_weather_merged.iloc[23,1]

prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag2')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag2')] = alberta_weather_merged.iloc[23,1]

prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag3')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag3')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag3')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag4')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag4')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag4')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag4')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag5')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag5')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag5')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag5')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_edmonton_lag5')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_edmonton_lag6')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_edmonton_lag7')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[7,  prediction_df.columns.get_loc('temp_edmonton_lag8')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[7,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[8,  prediction_df.columns.get_loc('temp_edmonton_lag9')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag10')] = alberta_weather_merged.iloc[23,1]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag11')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag12')] = alberta_weather_merged.iloc[23,1]

prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag13')] = alberta_weather_merged.iloc[23,1]

prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag14')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag15')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag16')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag17')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag18')] = alberta_weather_merged.iloc[23,1]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag19')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[4,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_edmonton_lag20')] = alberta_weather_merged.iloc[23,1]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[3,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[4,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_edmonton_lag21')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[2,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[3,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[4,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_edmonton_lag22')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[1,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[2,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[3,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[4,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[22, prediction_df.columns.get_loc('temp_edmonton_lag23')] = alberta_weather_merged.iloc[23,1]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[0,1]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[1,1]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[2,1]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[3,1]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[4,1]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[5,1]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[6,1]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[7,1]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[8,1]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[9,1]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[10,1]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[11,1]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[12,1]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[13,1]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[14,1]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[15,1]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[16,1]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[17,1]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[18,1]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[19,1]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[20,1]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[21,1]
prediction_df.iloc[22, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[22,1]
prediction_df.iloc[23, prediction_df.columns.get_loc('temp_edmonton_lag24')] = alberta_weather_merged.iloc[23,1]


prediction_df.iloc[:,24:24+24]

,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3,temp_edmonton_lag4,temp_edmonton_lag5,temp_edmonton_lag6,temp_edmonton_lag7,temp_edmonton_lag8,temp_edmonton_lag9,temp_edmonton_lag10,...,temp_edmonton_lag15,temp_edmonton_lag16,temp_edmonton_lag17,temp_edmonton_lag18,temp_edmonton_lag19,temp_edmonton_lag20,temp_edmonton_lag21,temp_edmonton_lag22,temp_edmonton_lag23,temp_edmonton_lag24
0,-2.0,-1.0,-1.0,0.0,1.0,3.0,4.0,4.0,4.0,4.0,...,0.0,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0
1,-6.0,-2.0,-1.0,-1.0,0.0,1.0,3.0,4.0,4.0,4.0,...,2.0,0.0,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0
2,-6.0,-6.0,-2.0,-1.0,-1.0,0.0,1.0,3.0,4.0,4.0,...,3.0,2.0,0.0,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0
3,-7.0,-6.0,-6.0,-2.0,-1.0,-1.0,0.0,1.0,3.0,4.0,...,3.0,3.0,2.0,0.0,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0
4,-6.0,-7.0,-6.0,-6.0,-2.0,-1.0,-1.0,0.0,1.0,3.0,...,4.0,3.0,3.0,2.0,0.0,-1.0,-2.0,-2.0,-3.0,-4.0
5,-4.0,-6.0,-7.0,-6.0,-6.0,-2.0,-1.0,-1.0,0.0,1.0,...,4.0,4.0,3.0,3.0,2.0,0.0,-1.0,-2.0,-2.0,-3.0
6,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,-2.0,-1.0,-1.0,0.0,...,4.0,4.0,4.0,3.0,3.0,2.0,0.0,-1.0,-2.0,-2.0
7,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,-2.0,-1.0,-1.0,...,4.0,4.0,4.0,4.0,3.0,3.0,2.0,0.0,-1.0,-2.0
8,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,-2.0,-1.0,...,4.0,4.0,4.0,4.0,4.0,3.0,3.0,2.0,0.0,-1.0
9,-1.0,-2.0,-2.0,-3.0,-4.0,-6.0,-7.0,-6.0,-6.0,-2.0,...,3.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,2.0,0.0


In [8]:
prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag1')] = alberta_weather_merged.iloc[23,0]

prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag2')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag2')] = alberta_weather_merged.iloc[23,0]

prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag3')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag3')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag3')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag4')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag4')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag4')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag4')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag5')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag5')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag5')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag5')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_calgary_lag5')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_calgary_lag6')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_calgary_lag7')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[7,  prediction_df.columns.get_loc('temp_calgary_lag8')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[1,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[2,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[3,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[4,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[5,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[6,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[7,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[8,  prediction_df.columns.get_loc('temp_calgary_lag9')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag10')] = alberta_weather_merged.iloc[23,0]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag11')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag12')] = alberta_weather_merged.iloc[23,0]

prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag13')] = alberta_weather_merged.iloc[23,0]

prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag14')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag15')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag16')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag17')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag18')] = alberta_weather_merged.iloc[23,0]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag19')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[4,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_calgary_lag20')] = alberta_weather_merged.iloc[23,0]




prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[3,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[4,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_calgary_lag21')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[2,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[3,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[4,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_calgary_lag22')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[1,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[2,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[3,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[4,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[22, prediction_df.columns.get_loc('temp_calgary_lag23')] = alberta_weather_merged.iloc[23,0]



prediction_df.iloc[0, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[0,0]
prediction_df.iloc[1, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[1,0]
prediction_df.iloc[2, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[2,0]
prediction_df.iloc[3, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[3,0]
prediction_df.iloc[4, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[4,0]
prediction_df.iloc[5, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[5,0]
prediction_df.iloc[6, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[6,0]
prediction_df.iloc[7, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[7,0]
prediction_df.iloc[8, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[8,0]
prediction_df.iloc[9, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[9,0]
prediction_df.iloc[10, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[10,0]
prediction_df.iloc[11, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[11,0]
prediction_df.iloc[12, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[12,0]
prediction_df.iloc[13, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[13,0]
prediction_df.iloc[14, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[14,0]
prediction_df.iloc[15, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[15,0]
prediction_df.iloc[16, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[16,0]
prediction_df.iloc[17, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[17,0]
prediction_df.iloc[18, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[18,0]
prediction_df.iloc[19, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[19,0]
prediction_df.iloc[20, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[20,0]
prediction_df.iloc[21, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[21,0]
prediction_df.iloc[22, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[22,0]
prediction_df.iloc[23, prediction_df.columns.get_loc('temp_calgary_lag24')] = alberta_weather_merged.iloc[23,0]


prediction_df.iloc[:,48:48+24]

,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_calgary_lag4,temp_calgary_lag5,temp_calgary_lag6,temp_calgary_lag7,temp_calgary_lag8,temp_calgary_lag9,temp_calgary_lag10,...,temp_calgary_lag15,temp_calgary_lag16,temp_calgary_lag17,temp_calgary_lag18,temp_calgary_lag19,temp_calgary_lag20,temp_calgary_lag21,temp_calgary_lag22,temp_calgary_lag23,temp_calgary_lag24
0,-2.0,-1.0,-1.0,0.0,1.0,1.0,2.0,3.0,3.0,4.0,...,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0
1,-4.0,-2.0,-1.0,-1.0,0.0,1.0,1.0,2.0,3.0,3.0,...,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0
2,-5.0,-4.0,-2.0,-1.0,-1.0,0.0,1.0,1.0,2.0,3.0,...,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0
3,-6.0,-5.0,-4.0,-2.0,-1.0,-1.0,0.0,1.0,1.0,2.0,...,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0
4,-5.0,-6.0,-5.0,-4.0,-2.0,-1.0,-1.0,0.0,1.0,1.0,...,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0
5,-3.0,-5.0,-6.0,-5.0,-4.0,-2.0,-1.0,-1.0,0.0,1.0,...,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0
6,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0,-2.0,-1.0,-1.0,0.0,...,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0
7,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0,-2.0,-1.0,-1.0,...,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0
8,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0,-2.0,-1.0,...,2.0,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0
9,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0,-2.0,...,1.0,2.0,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0


In [9]:
'''
#alberta_weather_merged_reversed = alberta_weather_merged.iloc[::-1]
count = 0
for i in range(24,24+24):
    for j in range(0,i-23):
        k = 
            prediction_df.iloc[j,i] = alberta_weather_merged.iloc[k,1]
        
prediction_df.iloc[:,24:24+24]
'''

'\n#alberta_weather_merged_reversed = alberta_weather_merged.iloc[::-1]\ncount = 0\nfor i in range(24,24+24):\n    for j in range(0,i-23):\n        k = \n            prediction_df.iloc[j,i] = alberta_weather_merged.iloc[k,1]\n        \nprediction_df.iloc[:,24:24+24]\n'

In [10]:
prediction_df.iloc[:,20:72] = prediction_df.iloc[:,20:72].astype('float64')

In [11]:
prediction_df

,year,day_of_month,month,day_of_year,hour_of_day,hour_of_the_week,hour_x_day,sin.hour.daily,cos.hour.daily,sin.hour.weekly,...,temp_calgary_lag15,temp_calgary_lag16,temp_calgary_lag17,temp_calgary_lag18,temp_calgary_lag19,temp_calgary_lag20,temp_calgary_lag21,temp_calgary_lag22,temp_calgary_lag23,temp_calgary_lag24
0,21,25,2,56,0,73,0,0.000000e+00,1.000000e+00,3.998920e-01,...,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0,-4.0
1,21,25,2,56,1,74,56,2.588190e-01,9.659258e-01,3.653410e-01,...,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0,-5.0
2,21,25,2,56,2,75,112,5.000000e-01,8.660254e-01,3.302791e-01,...,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0,-6.0
3,21,25,2,56,3,76,168,7.071068e-01,7.071068e-01,2.947552e-01,...,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0,-5.0
4,21,25,2,56,4,77,224,8.660254e-01,5.000000e-01,2.588190e-01,...,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0,-3.0
5,21,25,2,56,5,78,280,9.659258e-01,2.588190e-01,2.225209e-01,...,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0,-2.0
6,21,25,2,56,6,79,336,1.000000e+00,6.123234e-17,1.859116e-01,...,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0,-2.0
7,21,25,2,56,7,80,392,9.659258e-01,-2.588190e-01,1.490423e-01,...,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0,-1.0
8,21,25,2,56,8,81,448,8.660254e-01,-5.000000e-01,1.119645e-01,...,2.0,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0,-1.0
9,21,25,2,56,9,82,504,7.071068e-01,-7.071068e-01,7.473009e-02,...,1.0,2.0,3.0,3.0,4.0,3.0,3.0,2.0,1.0,0.0


In [12]:
for i in range(0,len(prediction_df.columns)):
    print(i, '--->',prediction_df.columns[i],'--->', prediction_df.dtypes[i])

0 ---> year ---> int64
1 ---> day_of_month ---> int64
2 ---> month ---> int64
3 ---> day_of_year ---> int64
4 ---> hour_of_day ---> int64
5 ---> hour_of_the_week ---> int64
6 ---> hour_x_day ---> int64
7 ---> sin.hour.daily ---> float64
8 ---> cos.hour.daily ---> float64
9 ---> sin.hour.weekly ---> float64
10 ---> cos.hour.weekly ---> float64
11 ---> sin.day.yearly ---> float64
12 ---> cos.day.yearly ---> float64
13 ---> weekend ---> int64
14 ---> monday ---> int64
15 ---> tuesday ---> int64
16 ---> wednesday ---> int64
17 ---> thursday ---> int64
18 ---> friday ---> int64
19 ---> saturday ---> int64
20 ---> sunday ---> float64
21 ---> temp_calgary ---> float64
22 ---> temp_edmonton ---> float64
23 ---> is_Covid ---> float64
24 ---> temp_edmonton_lag1 ---> float64
25 ---> temp_edmonton_lag2 ---> float64
26 ---> temp_edmonton_lag3 ---> float64
27 ---> temp_edmonton_lag4 ---> float64
28 ---> temp_edmonton_lag5 ---> float64
29 ---> temp_edmonton_lag6 ---> float64
30 ---> temp_edmonton_lag

In [13]:
prediction_df.to_csv('input_data_{}_{}_{}_generate_date_{}_v11.csv'.format(day_of_run, month_of_run, year_of_run, now_time))

In [14]:

#Generate the forecast


loaded_model = joblib.load("model_generated_on_17_2_2021_v11.joblib.dat")


forecast = pd.DataFrame() # this dataframe will contain the forecasts (timestamp and values)
forecast['datetime_of_forecast']= forecast_interval_stamps.values 
forecast['values']              = 0 #initializing the column with zero

for i in range(0, 24):
    input_data          = prediction_df.iloc[i]
    predictions         = loaded_model.predict(np.array(input_data).reshape((1,-1)))
    forecast.iloc[i,1]  = predictions[0]


forecast



#forecast.to_csv('forecast_{}_{}_{}_generate_date_{}.csv'.format(day_of_run, month_of_run, year_of_run, now_time ))

,datetime_of_forecast,values
0,2021-02-25 00:00:00,9524.979492
1,2021-02-25 01:00:00,9494.361328
2,2021-02-25 02:00:00,9507.480469
3,2021-02-25 03:00:00,9529.068359
4,2021-02-25 04:00:00,9471.387695
5,2021-02-25 05:00:00,9616.150391
6,2021-02-25 06:00:00,10005.597656
7,2021-02-25 07:00:00,10393.093750
8,2021-02-25 08:00:00,10451.216797
9,2021-02-25 09:00:00,10435.131836


In [15]:
'''
datetime_of_forecast	values
0	2021-02-14 00:00:00	10466.520508
1	2021-02-14 01:00:00	10438.880859
2	2021-02-14 02:00:00	10402.455078
3	2021-02-14 03:00:00	10428.241211
4	2021-02-14 04:00:00	10550.427734
5	2021-02-14 05:00:00	10511.577148
6	2021-02-14 06:00:00	10533.135742
7	2021-02-14 07:00:00	10722.361328
8	2021-02-14 08:00:00	10906.462891
9	2021-02-14 09:00:00	11113.807617
10	2021-02-14 10:00:00	11054.803711
11	2021-02-14 11:00:00	11118.104492
12	2021-02-14 12:00:00	11045.169922
13	2021-02-14 13:00:00	10944.636719
14	2021-02-14 14:00:00	10860.379883
15	2021-02-14 15:00:00	10931.954102
16	2021-02-14 16:00:00	11139.295898
17	2021-02-14 17:00:00	11391.276367
18	2021-02-14 18:00:00	11468.875977
19	2021-02-14 19:00:00	11430.413086
20	2021-02-14 20:00:00	11289.504883
21	2021-02-14 21:00:00	11146.918945
22	2021-02-14 22:00:00	10916.745117
23	2021-02-14 23:00:00	10707.790039




datetime_of_forecast	values
0	2021-02-14 00:00:00	10450.235352
1	2021-02-14 01:00:00	10417.850586
2	2021-02-14 02:00:00	10414.645508
3	2021-02-14 03:00:00	10521.308594
4	2021-02-14 04:00:00	10401.457031
5	2021-02-14 05:00:00	10389.750000
6	2021-02-14 06:00:00	10536.349609
7	2021-02-14 07:00:00	10759.925781
8	2021-02-14 08:00:00	10851.803711
9	2021-02-14 09:00:00	11015.231445
10	2021-02-14 10:00:00	11117.164062
11	2021-02-14 11:00:00	11227.749023
12	2021-02-14 12:00:00	11118.197266
13	2021-02-14 13:00:00	11031.517578
14	2021-02-14 14:00:00	10946.366211
15	2021-02-14 15:00:00	11008.471680
16	2021-02-14 16:00:00	11219.875977
17	2021-02-14 17:00:00	11381.377930
18	2021-02-14 18:00:00	11383.379883
19	2021-02-14 19:00:00	11365.541016
20	2021-02-14 20:00:00	11356.795898
21	2021-02-14 21:00:00	11250.417969
22	2021-02-14 22:00:00	10993.984375
23	2021-02-14 23:00:00	10685.560547


datetime_of_forecast	values
0	2021-02-14 00:00:00	10619.396484
1	2021-02-14 01:00:00	10527.563477
2	2021-02-14 02:00:00	10498.200195
3	2021-02-14 03:00:00	10499.145508
4	2021-02-14 04:00:00	10675.842773
5	2021-02-14 05:00:00	10667.277344
6	2021-02-14 06:00:00	10831.541992
7	2021-02-14 07:00:00	10855.270508
8	2021-02-14 08:00:00	10972.903320
9	2021-02-14 09:00:00	11059.010742
10	2021-02-14 10:00:00	11046.155273
11	2021-02-14 11:00:00	11032.979492
12	2021-02-14 12:00:00	11032.975586
13	2021-02-14 13:00:00	11025.170898
14	2021-02-14 14:00:00	10999.747070
15	2021-02-14 15:00:00	10988.877930
16	2021-02-14 16:00:00	11097.058594
17	2021-02-14 17:00:00	11231.550781
18	2021-02-14 18:00:00	11308.552734
19	2021-02-14 19:00:00	11303.566406
20	2021-02-14 20:00:00	11208.306641
21	2021-02-14 21:00:00	11071.291016
22	2021-02-14 22:00:00	10965.165039
23	2021-02-14 23:00:00	10684.395508

'''



'\ndatetime_of_forecast\tvalues\n0\t2021-02-14 00:00:00\t10466.520508\n1\t2021-02-14 01:00:00\t10438.880859\n2\t2021-02-14 02:00:00\t10402.455078\n3\t2021-02-14 03:00:00\t10428.241211\n4\t2021-02-14 04:00:00\t10550.427734\n5\t2021-02-14 05:00:00\t10511.577148\n6\t2021-02-14 06:00:00\t10533.135742\n7\t2021-02-14 07:00:00\t10722.361328\n8\t2021-02-14 08:00:00\t10906.462891\n9\t2021-02-14 09:00:00\t11113.807617\n10\t2021-02-14 10:00:00\t11054.803711\n11\t2021-02-14 11:00:00\t11118.104492\n12\t2021-02-14 12:00:00\t11045.169922\n13\t2021-02-14 13:00:00\t10944.636719\n14\t2021-02-14 14:00:00\t10860.379883\n15\t2021-02-14 15:00:00\t10931.954102\n16\t2021-02-14 16:00:00\t11139.295898\n17\t2021-02-14 17:00:00\t11391.276367\n18\t2021-02-14 18:00:00\t11468.875977\n19\t2021-02-14 19:00:00\t11430.413086\n20\t2021-02-14 20:00:00\t11289.504883\n21\t2021-02-14 21:00:00\t11146.918945\n22\t2021-02-14 22:00:00\t10916.745117\n23\t2021-02-14 23:00:00\t10707.790039\n\n\n\n\ndatetime_of_forecast\tvalues\n0\t